# Машинное обучение, DS-поток, осень 2022
## Задание 3.10


**Правила:**

* Дедлайны см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно **ноутбук в формате `ipynb`**.
* Следите за размером файлов. **Бот не может принимать файлы весом более 20 Мб.** Если файл получается больше, заранее разделите его на несколько.
* Выполнять задание необходимо полностью самостоятельно. **При обнаружении списывания все участники списывания будут сдавать устный зачет.**
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Кроме того, публикация решения в открытом доступе может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него. Можно добавлять необходимое количество ячеек.
* Комментарии к решению пишите в markdown-ячейках.
* Выполнение задания (ход решения, выводы и пр.) должно быть осуществлено на русском языке.
* Если код будет не понятен проверяющему, оценка может быть снижена.
* Никакой код из данного задания при проверке запускаться не будет. *Если код студента не выполнен, недописан и т.д., то он не оценивается.*
* В каждой задаче не забывайте делать **пояснения и выводы**.


**Правила оформления теоретических задач:**

В задании требуется вывести формулы для обратного прохода.
* Решения необходимо прислать одним из следующих способов:
  * фотографией в правильной ориентации, где все четко видно, а почерк разборчив,
    * отправив ее как файл боту вместе с ноутбуком *или*
    * вставив ее в ноутбук посредством `Edit -> Insert Image` (<font color="red">фото, вставленные ссылкой, не принимаются</font>);
  * в виде $LaTeX$ в markdown-ячейках.
* Решения не проверяются, если какое-то требование не выполнено. Особенно внимательно все проверьте в случае выбора второго пункта (вставки фото в ноутбук). <font color="red"><b>Неправильно вставленные фотографии могут не передаться при отправке.</b></font> Для проверки попробуйте переместить `ipynb` в другую папку и открыть его там.
* В решениях поясняйте, чем вы пользуетесь, хотя бы кратко. Например, если пользуетесь независимостью, то достаточно подписи вида "*X и Y незав.*"
* Решение, в котором есть только ответ, и отсутствуют вычисления, оценивается в 0 баллов

**Баллы за задание:**

* Применение модулей на синтетических данных **`0_train_model.ipynb`** — **55 баллов**.
* Реализация модулей **`1_modules.ipynb`** — **100 баллов**.
* Исследование проcтых нейронных сетей **`2_research_model.ipynb`** — **X баллов**.

<h1 align=center>
    Добро пожаловать в Глубокое обучение!
</h1>

<img src="https://www.analyticsinsight.net/wp-content/uploads/2019/01/AutoML-1024x695.jpg" width=500>

В этом домашнем задании вам предстоит самостоятельно реализовать самые важные компоненты нейросети, **используя только `NumPy`**. 

Ноутбуков в задании много (целых 3), однако большая часть кода написана за вас.😉 Вам предстоит лишь  разобраться в уже написанном коде, понять, как работют некоторые важные компоненты нейронных сетей и написать недстающие строки кода, чтобы все заработало.

Структура задания подразумевает выполнение 3-х `jupyter notebook`'ов.

**`0_train_model.ipynb`**: текущий ноутбук. В нем поясняется суть задания, описаны остальные ноутбуки и именно этот ноутбук нужно запустить, когда все слои уже реализованы.  
**`1_modules.ipynb`**: ноутбук, в котором нужно непосредственно реализовать слои нейронной сети, используя только библиотеку NumPy.  
**`2_research_model.ipynb`**: ноутбук независимый от предыдущих трех, где нужно будет произвести исследование простой нейронной сети.

*Задание на следующей неделе будет похожим на это. Нужно будет реализовть чуть более сложные модули и применить их для создания более продвинутой модели.*

***
## Применение реализованных модулей (55 баллов)
Для начала определим путь до ноутбука с реализованными модулями.

In [ ]:
# Путь к папке с заданием
path_to_dir: str = 'Загрузки/'
# Путь к файлу с модулями
path_to_modules_notebook = path_to_dir + "1_modules.ipynb"

In [ ]:
# В Google Colab подключаемся к диску
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')
    path_to_modules_notebook = "drive/MyDrive/" + path_to_modules_notebook

Следующая ячейка делает `from 1_modules.ipynb import *`, таким образом позволяя этому ноутбуку видеть все модули.

In [ ]:
%run $path_to_modules_notebook

После того, как все слои в `1_modules.ipynb` протестированы и работают корректно, проверим нейросеть на наборах синтетических данных.

In [ ]:
from IPython import display
from itertools import cycle, islice
import time
from time import time, sleep
import warnings

import numpy as np
import pandas as pd
import scipy.stats as sps

from sklearn import cluster, datasets
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

import torch
import torch.nn as nn
from torch.optim import SGD as torch_sgd


cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF3300', '#00CC66'])
RANDOM_SEED = 42

sns.set(font_scale=1.5, palette='Set2')
np.random.seed(RANDOM_SEED)

Будем применять реализованные модули на синтетических данных.

### 1. Линейно разделимая выборка (30 баллов)

Обучим однослойную нейронную сеть решать линейно-разделимую классификацию на 2 класса.

In [ ]:
n_samples = 1024
n_classes = 2

varied_blobs = datasets.make_blobs(
    n_samples=n_samples,
    n_features=2,
    centers=n_classes,
    cluster_std=[1.0, 2.5], 
    random_state=RANDOM_SEED
)

sample, labels = varied_blobs

In [ ]:
plt.figure(figsize=(12,7))
plt.title('Линейно разделимая выборка')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')
plt.scatter(sample[:,0], sample[:,1], c=labels, cmap=cm_bright, alpha=0.55);

Генератор батчей. Помним, что нейросети обучаются итеративно $-$ по батчам.

In [ ]:
def train_generator(sample, labels, batch_size):
    '''
    Генератор батчей. 
    На каждом шаге возвращает `batch_size` объектов из `sample` и их 
    меток из `labels`.
    '''
    n_samples = sample.shape[0]
    # Перемешиваем в случайном порядке в начале эпохи
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    # Обратите внимание на yield вместо return
    # (если раньше не сталкивались с генераторами)
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        batch_idx = indices[start:end]
        yield sample[batch_idx], labels[batch_idx]

Функция для удобного обучения модели.

In [ ]:
def train_model(
    model, 
    sample, y,
    criterion,
    opt_params, 
    opt_state, 
    n_epoch, 
    batch_size
):
    '''
    Обучает модель из вашего мини-фреймворка. 
    Возвращает обученную модель, историю значений функции потерь 
    и метрики качества.
    
    :param `model`: модель из вашего мини-фреймворка
    :param `sample`: матрица объектов
    :param `y`: вектор истинных меток объектов
    :param `criterion`: функция потерь
    :param `opt_params`: гиперпараметры оптимизатора
    :param `opt_state`: текущая информация о градиентах, 
                        хранящаяся в оптимизаторе
    :param `n_epoch`: количество эпох
    :param `batch_size`: размер одного батча
    '''
    
    loss_history = []

    for i in range(n_epoch):
        for x_batch, y_batch in train_generator(sample, y, batch_size):
            # Обнуляем градиенты с предыдущей итерации
            # Forward pass
            # Backward pass
            
            <ВАШ КОД ЗДЕСЬ>
            
            # Обновление весов
            SGD(model.get_parameters(), 
                model.get_grad_params(), 
                opt_params,
                opt_state)
            loss_history.append(loss)

        display.clear_output(wait=True)
        
        plt.figure(figsize=(8, 6))
        plt.title("Функция потерь на train")
        plt.xlabel("итерация")
        plt.ylabel("лосс")
        plt.plot(loss_history, 'b')
        plt.title(f'Loss: {loss:.3f}')
        plt.show()
    
    return model, loss_history

Построим однослойную нейросеть для классификации: размер слоя $2 \times 2$, так как 2 признака и 2 класса. В качестве последнего слоя рекомендуется использовать `LogSoftMax`.

In [ ]:
net = <ВАШ КОД ЗДЕСЬ>

print(net)

Объявим оптимизируемую функцию потерь и гиперпаарметры.

In [ ]:
# Функция потерь
criterion = <ВАШ КОД ЗДЕСЬ>

# Гиперпараметры оптимизатора
optimizer_config = {'learning_rate' : 1e-2}
optimizer_state = {}

# Гиперпараметры цикла обучения и генератора
n_epoch = 50
batch_size = 128

Проверим, что кодирование производится верно.

In [ ]:
y = np.hstack([1-labels[:,None], labels[:,None]])
print(y.shape)
print(labels[:10])
print(y[:10])

Обучим модель с помощью функции `train_model`.

In [ ]:
net, loss_history = train_model(
    net, 
    sample, y, 
    criterion,
    optimizer_config, 
    optimizer_state,
    n_epoch,
    batch_size
)

Поменяйте гиперпараметр `learning_rate` в `optimizer_config` на 1е-1, 1е-2, 1е-3 и 1е-4. 

Как это влияет на обучение?

Батч-генератор для тестовой выборки.

In [ ]:
def generate_grid(sample, h=0.02):
    '''
    Генерирует двумерную сетку.
    '''
    x_min, x_max = sample[:, 0].min() - .5, sample[:, 0].max() + .5
    y_min, y_max = sample[:, 1].min() - .5, sample[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    return xx, yy


def test_generator(sample):
    '''
    Батч-генератор для тестовых данных (без меток).
    '''
    n_samples = sample.shape[0]
    indices = np.arange(n_samples)
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        batch_idx = indices[start:end]
        yield sample[batch_idx]

Функции для удобного тестирования модели.

In [ ]:
def test_model(model, test_sample):
    '''
    Тестирует модель на тестовой выборке.
    '''
    preds = []
    for test_batch in test_generator(test_sample):
        batch = model.forward(test_batch)
        batch = batch.argmax(axis=1).reshape(-1,1)
        preds.append(batch)
    preds = np.vstack(preds)
    return preds

In [ ]:
def plot_grid_preds(sample, labels, xx, yy, preds, title):
    '''
    Функция для удобной отрисовки предсказаний нейросети на двумерной сетке.
    '''
    plt.figure(figsize=(12,7))
    plt.title(title)
    plt.xlabel('Признак 1')
    plt.ylabel('Признак 2')
    plt.scatter(sample[:,0], sample[:,1], c=labels, cmap=cm_bright, alpha=0.55)
    plt.contourf(xx, yy, preds, alpha=.2, cmap=cm)
    plt.show();

In [ ]:
xx, yy = <ВАШ КОД ЗДЕСЬ>
test_sample = <ВАШ КОД ЗДЕСЬ>

test_predictions = <ВАШ КОД ЗДЕСЬ>
test_predictions = <ВАШ КОД ЗДЕСЬ>
   
plot_grid_preds(
    sample, 
    labels, 
    xx, yy, 
    test_predictions, 
    'Линейно разделимая выборка'
)

Сравним с точно такой же моделью, но на PyTorch.

*Подсказка.* Названия слоев идентичны, если брать их из `torch.nn`. Отличия только в оптимизаторе и названии функции потерь.

In [ ]:
net_torch = <ВАШ КОД ЗДЕСЬ>

print(net_torch)

In [ ]:
def train_model_torch(
    model, 
    sample, y,
    criterion,
    optimizer,
    n_epoch, 
    batch_size
):
    '''
    Обучает модель из PyTorch. 
    Возвращает обученную модель, историю значений функции потерь 
    и метрики качества.
    
    :param `model`: модель из PyTorch
    :param `sample`: матрица объектов
    :param `y`: вектор истинных меток объектов
    :param `criterion`: функция потерь
    :param `opt_params`: гиперпараметры оптимизатора
    :param `opt_state`: текущая информация о градиентах, 
                        хранящаяся в оптимизаторе
    :param `n_epoch`: количество эпох
    :param `batch_size`: размер одного батча
    '''
    
    loss_history = []

    for i in range(n_epoch):
        for x_batch, y_batch in train_generator(sample, y, batch_size):
            # Обнуляем градиенты с предыдущей итерации
            # Forward pass
            # Backward pass
            # Обновление весов
            <ВАШ КОД ЗДЕСЬ>
            
            loss_history.append(loss.data)

        display.clear_output(wait=True)
        
        plt.figure(figsize=(8, 6))
        plt.title("Функция потерь на train")
        plt.xlabel("итерация")
        plt.ylabel("лосс")
        plt.plot(loss_history, 'b')
        plt.show()
        print('Current loss: %f' % loss)
    
    return model, loss_history

In [ ]:
criterion = <ВАШ КОД ЗДЕСЬ>
optimizer = torch_sgd(<ВАШ КОД ЗДЕСЬ>)

net_torch, loss_history = train_model_torch(
    net_torch, sample, y, 
    criterion,
    optimizer,
    n_epoch,
    batch_size
)

*Для самопроверки.* Значение функции потерь (лосса) должны быть примерно одинаковые у вашей модели и у сети из PyTorch. Каким именно на этом датасете $-$ см таблицу ниже.

|  Архитектура  | learning_rate | Значение лосса |
|-------------|---------------|-------------|
| Linear(2,2) |  1e-2         | 0.1 и ниже |

In [ ]:
def test_torch(model, test_sample):
    '''
    Тестирует модель из PyTorch на тестовой выборке.
    '''
    preds = []
    with torch.no_grad():
        for test_batch in test_generator(test_sample):
            preds_batch = model(torch.FloatTensor(test_batch))
            preds_batch = preds_batch.argmax(axis=1).reshape(-1,1)
            preds.append(preds_batch.detach().numpy())
    preds = np.vstack(preds)
    return preds

In [ ]:
xx, yy = <ВАШ КОД ЗДЕСЬ>
test_sample = <ВАШ КОД ЗДЕСЬ>

test_predictions = <ВАШ КОД ЗДЕСЬ>
test_predictions = <ВАШ КОД ЗДЕСЬ>
   
plot_grid_preds(
    sample, 
    labels, 
    xx, yy, 
    test_predictions, 
    'Линейно разделимая выборка'
)

Рисунки с предсказаниями должны практически совпадать у вашей модели и у модели на PyTorch.

### 2. Вложенные окружности (20 баллов)

Проверим работу нейросети на более сложной выборке, которая линейно не разделима в исходном пространстве признаков.

In [ ]:
n_samples = 1024

noisy_circles = datasets.make_circles(
    n_samples=n_samples, 
    factor=.5, 
    noise=.05
)

sample, labels = noisy_circles

In [ ]:
plt.figure(figsize=(12,7))
plt.title('Вложенные окружности')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')
plt.scatter(sample[:,0], sample[:,1], c=labels, cmap=cm_bright, alpha=0.55);

Сначала попробуем обучить и протестировать однослойную неросеть:

In [ ]:
net = <ВАШ КОД ЗДЕСЬ>
print(net)

# Функция потерь
criterion = <ВАШ КОД ЗДЕСЬ>

# Функция потерь
criterion = NLLCriterion()

# Гиперпараметры оптимизатора
optimizer_config = {'learning_rate' : 1e-2}
optimizer_state = {}

# Гиперпараметры цикла обучения и генератора
n_epoch = 50
batch_size = 128

In [ ]:
y = np.hstack([1-labels[:,None], labels[:,None]])
print(y.shape)
print(labels[:10])
print(y[:10])

In [ ]:
net, loss_history = <ВАШ КОД ЗДЕСЬ>

In [ ]:
<ВАШ КОД ЗДЕСЬ>

plot_grid_preds(sample, labels, xx, yy, 
                test_predictions, 'Вложенные окружности')

Вряд ли у вас получился лосс ниже 0.68 и адекватный рисунок. Это говорит о том, что один слой не может решить задачу.

Попробуем сделать **двухслойную** нейросеть. Не жалейте нейронов в скрытый слой.

*Подсказка.* Не забудьте про нелинейности.

In [ ]:
net = <ВАШ КОД ЗДЕСЬ>
print(net)

criterion = <ВАШ КОД ЗДЕСЬ>

optimizer_config = {'learning_rate' : 1e-2}
optimizer_state = {}

n_epoch = 100
batch_size = 128

y = np.hstack([1-labels[:,None], labels[:,None]])
print(y.shape)

In [ ]:
net, loss_history = <ВАШ КОД ЗДЕСЬ>

In [ ]:
<ВАШ КОД ЗДЕСЬ>

plot_grid_preds(sample, labels, xx, yy, test_predictions, 'Вложенные окружности')

Попробуйте менять количество нейронов в скрытом слое. Начиная с какого количества нейронов окружности начинают хорошо разделяться?

И снова сравним с PyTorch.

In [ ]:
net_torch = <ВАШ КОД ЗДЕСЬ>

print(net_torch)

criterion = <ВАШ КОД ЗДЕСЬ>
optimizer = torch_sgd(<ВАШ КОД ЗДЕСЬ>)

net_torch, loss_history = <ВАШ КОД ЗДЕСЬ>

In [ ]:
<ВАШ КОД ЗДЕСЬ>

plot_grid_preds(sample, labels, xx, yy, 
                test_predictions, 'Вложенные окружности')

*Для самопроверки*. Значение функции потерь (лосса) должны быть примерно одинаковые у вашей модели и у сети из PyTorch. Каким именно на этом датасете $-$ см. таблицу ниже.

|  Архитектура  | learning_rate | Значение лосса |
|-------------|---------------|-------------|
| (2,100) -> ReLU -> (100,2) |  1e-2         | 0.45 и ниже |

Оба рисунка должны примерно совпадать и адекватно разделять окружности между собой (одна область должна быть "внутри" другой).

### 3. Заключение (5 баллов)

Сделайте выводы по проведенным экспериментам.

**Выводы.** ...

### Полезные ссылки

- [Заметки главы ИИ в Tesla Andrej Karpathy по обучению нейросетей](http://karpathy.github.io/2019/04/25/recipe/)
- [Backpropagation на brilliant.org](https://brilliant.org/wiki/backpropagation/)
- См. ссылки в `1_modules.ipynb`